<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Improving Your Knowledge Base</h1>

Imagine you've built and deployed an LLM question-answering service that enables users to ask questions and receive answers from a knowledge base. You want to understand what kinds of questions your users are asking and whether you're providing good answers to those questions.

Phoenix helps you pinpoint user queries that are not answered by your knowledge base so that you know which topics to iterate and improve upon. As you'll see, your users are asking questions on several topics that your knowledge base does not cover.

In this tutorial, you will:

- Download an pre-indexed knowledge base and run a LlamaIndex application
- Download user query data and knowledge base data, including embeddings computed using the OpenAI API
- Define a schema to describe the format of your data
- Launch Phoenix to visually explore your embeddings
- Investigate clusters of user queries with no corresponding knowledge base entry

⚠️ This notebook requires an [OpenAI API key](https://platform.openai.com/account/api-keys).

Let's get started!

## Install Dependencies and Import Libraries

Install Phoenix and LlamaIndex, a package that provides high-level APIs for LLM-powered applications such as question-answering services over knowledge bases.

In [ ]:
!pip install arize-phoenix llama-index

Import libraries.

In [ ]:
import textwrap
import zipfile
import os
import tempfile

import urllib
import pandas as pd
import phoenix as px
from langchain import OpenAI
from llama_index import StorageContext, load_index_from_storage
from llama_index.response.schema import Response

## What is Knowledge Base?

TODO: short explanation with diagram 

## Download Your Knowledge Base

Download and unzip a pre-built knowledge base index.

In [ ]:
def download_file(url: str, output_path: str) -> None:
    """
    Downloads a file from the specified URL and saves to a local path.
    """
    urllib.request.urlretrieve(url, output_path)


def unzip_directory(zip_path: str, output_path: str) -> None:
    """
    Unzips a directory to a specified output path.
    """
    with zipfile.ZipFile(zip_path, "r") as f:
        f.extractall(output_path)


print("⏳ Downloading knowledge base...")
data_dir = tempfile.gettempdir()
zip_file_path = os.path.join(data_dir, "database_index.zip")
download_file(
    url="http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/llama-index/database_index.zip",
    output_path=zip_file_path,
)

print("⏳ Unzipping knowledge base...")
index_dir = os.path.join(data_dir, "database_index")
unzip_directory(zip_file_path, index_dir)

print("✅ Done")

## Run Your Question-Answering Service

Set your OpenAI API key. You have two options here. You can either:
- set the `OPENAI_API_KEY` environment variable in your notebook environment
- set the `openai_api_key` variable below

In [ ]:
openai_api_key = None  # change this to your OpenAI API key if you do not have the OPENAI_API_KEY environment variable set

Start a LlamaIndex application from your pre-computed index.

In [ ]:
storage_context = StorageContext.from_defaults(
    persist_dir=index_dir,
)
llm = OpenAI(temperature=0, model_name="gpt-4", openai_api_key=openai_api_key)
index = load_index_from_storage(storage_context, llm=llm)
query_engine = index.as_query_engine()

Display a list of topics covered by your knowledge base. TODO

Define a function to display both the response from the service in addition to the retrieved knowledge base entry. By default, LlamaIndex retrieves the two most similar entries to the query by cosine similarity.

In [ ]:
def display_llama_index_response(response: Response) -> None:
    """
    Displays a LlamaIndex response and its source nodes (retrieved context).
    """

    print("Response")
    print("========")
    for line in textwrap.wrap(response.response.strip(), width=80):
        print(line)
    print()

    print("Source Nodes")
    print("============")
    print()

    for source_node in response.source_nodes:
        print(f"doc_id: {source_node.node.doc_id}")
        print(f"score: {source_node.score}")
        print()
        for line in textwrap.wrap(source_node.node.text, width=80):
            print(line)
        print()

Ask a question of your question-answering service. See the response in addition to the retrieved context from your knowledge base.

In [ ]:
query = "What is the name of the character Microsoft used to make Windows 8 seem more personable?"
response = query_engine.query(query)
display_llama_index_response(response)

## Download and Inspect Your Data

Download knowledge base (database) and user query data.

In [ ]:
database_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/llama-index/database.parquet"
)
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/llama-index/query.parquet"
)

View a few database rows.

In [ ]:
database_df.head()

The fields of the dataframe are: TODO

View a few query rows.

In [ ]:
query_df.head()

The fields of the dataframe are: TODO

## Launch Phoenix

Define a schema to tell Phoenix what the columns of your training dataframe represent (features, predictions, actuals, tags, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [ ]:
schema = px.Schema(
    embedding_feature_column_names={
        "text_embedding": px.EmbeddingColumnNames(
            vector_column_name="text_vector",
            raw_data_column_name="text",
        )
    },
    tag_column_names=["granular_subject", "broad_subject", "is_answerable"],
)

Create Phoenix datasets that wrap your dataframes with schemas that describe them.

In [ ]:
database_ds = px.Dataset(database_df, schema, name="database")
query_ds = px.Dataset(query_df, schema, name="query")

Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI.

In [ ]:
session = px.launch_app(primary=query_ds, reference=database_ds)

## Investigate User Interests and Improve Your Knowledge Base

TODO: write this section

Congrats! You've found the topics your users are asking about that are not covered in your knowledge base. As an actionable next step, you can augment your knowledge base to cover these topics and improve your users' experience.